In [1]:
import numpy as np
import pandas as pd

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from xgboost import XGBRegressor
import numpy as np
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
df_train = pd.read_csv('./training_data.csv')
df_test = pd.read_csv('./testing_data.csv')

In [4]:
df_train.head()

,ID,timestamp,cpu_type,cpu_limit,ram_limit,cpu_usage,ram_usage,num_req,conc_lvl,latency
0,2714,2021-11-04 23:15:38,Intel Xeon Gold 5317,2.0,70M,0.084450,46854144.0,100,71,7429000
1,38113,2021-11-04 23:23:27,AMD EPYC 72F3,2.0,70M,0.277893,55934976.0,300,21,2196000
2,33409,2021-11-11 13:32:59,Intel Xeon Platinum 8380,2.5,66M,0.351042,59109376.0,700,1,37000
3,10123,2022-03-19 02:35:05,Intel Xeon Platinum 8356H,0.8,768M,0.026230,43659264.0,100,41,3946000
4,12815,2022-03-17 06:30:29,AMD EPYC 72F3,0.4,512M,0.108346,51048448.0,800,51,5084000


In [5]:
df_train.columns

Index(['ID', 'timestamp', 'cpu_type', 'cpu_limit', 'ram_limit', 'cpu_usage',
       'ram_usage', 'num_req', 'conc_lvl', 'latency'],
      dtype='object')

In [6]:
print(df_train.info())
print(df_train.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41134 entries, 0 to 41133
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         41134 non-null  int64  
 1   timestamp  41134 non-null  object 
 2   cpu_type   41134 non-null  object 
 3   cpu_limit  41134 non-null  float64
 4   ram_limit  41134 non-null  object 
 5   cpu_usage  41134 non-null  float64
 6   ram_usage  41134 non-null  float64
 7   num_req    41134 non-null  int64  
 8   conc_lvl   41134 non-null  int64  
 9   latency    41134 non-null  int64  
dtypes: float64(3), int64(4), object(3)
memory usage: 3.1+ MB
None
                 ID     cpu_limit     cpu_usage     ram_usage       num_req  \
count  41134.000000  41134.000000  41134.000000  4.113400e+04  41134.000000   
mean   29357.337434      1.445007      0.205489  5.377951e+07    532.958137   
std    16909.992230      1.077288      0.141493  7.771771e+06    286.221513   
min        0.000000      0.1

In [7]:
df_train.shape

(41134, 10)

In [8]:
df_train.dropna(inplace=True)

In [9]:
df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
df_train['hour'] = df_train['timestamp'].dt.hour
df_train['dayofweek'] = df_train['timestamp'].dt.dayofweek

In [10]:
df_train['log_cpu_usage'] = np.log1p(df_train['cpu_usage'])
df_train['log_ram_usage'] = np.log1p(df_train['ram_usage'])
df_train['log_num_req'] = np.log1p(df_train['num_req'])
df_train['log_conc_lvl'] = np.log1p(df_train['conc_lvl'])

In [11]:
df_train

,ID,timestamp,cpu_type,cpu_limit,ram_limit,cpu_usage,ram_usage,num_req,conc_lvl,latency,hour,dayofweek,log_cpu_usage,log_ram_usage,log_num_req,log_conc_lvl
0,2714,2021-11-04 23:15:38,Intel Xeon Gold 5317,2.0,70M,0.084450,46854144.0,100,71,7429000,23,3,0.081073,17.662550,4.615121,4.276666
1,38113,2021-11-04 23:23:27,AMD EPYC 72F3,2.0,70M,0.277893,55934976.0,300,21,2196000,23,3,0.245213,17.839700,5.707110,3.091042
2,33409,2021-11-11 13:32:59,Intel Xeon Platinum 8380,2.5,66M,0.351042,59109376.0,700,1,37000,13,3,0.300876,17.894900,6.552508,0.693147
3,10123,2022-03-19 02:35:05,Intel Xeon Platinum 8356H,0.8,768M,0.026230,43659264.0,100,41,3946000,2,5,0.025892,17.591926,4.615121,3.737670
4,12815,2022-03-17 06:30:29,AMD EPYC 72F3,0.4,512M,0.108346,51048448.0,800,51,5084000,6,3,0.102869,17.748286,6.685861,3.951244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41129,54343,2022-03-18 02:05:09,Intel Xeon Silver 4309Y,0.7,896M,0.169227,50458624.0,900,11,1039000,2,4,0.156343,17.736664,6.803505,2.484907
41130,38158,2022-03-11 14:24:38,Intel Core i9-11900K,1.5,640M,0.147585,50913280.0,300,1,53000,14,4,0.137660,17.745634,5.707110,0.693147
41131,860,2022-03-16 16:28:38,Intel Xeon Platinum 8352M,1.8,640M,0.134895,51859456.0,900,31,3145000,16,2,0.126540,17.764048,6.803505,3.465736
41132,15795,2022-03-17 03:18:29,Intel Xeon Platinum 8368,0.7,768M,0.203862,45862912.0,800,21,2001000,3,3,0.185535,17.641167,6.685861,3.091042


In [12]:
df_train.drop(columns=['timestamp'], inplace=True)

In [13]:
def extract_numeric_ram_limit(ram_limit):
    return float(ram_limit.replace('M', ''))

In [14]:
df_train['ram_limit_numeric'] = df_train['ram_limit'].apply(extract_numeric_ram_limit)
df_test['ram_limit_numeric'] = df_test['ram_limit'].apply(extract_numeric_ram_limit)

In [15]:
df_train.drop(columns=['ram_limit'], inplace=True)
df_test.drop(columns=['ram_limit'], inplace=True)

In [16]:
print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41134 entries, 0 to 41133
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 41134 non-null  int64  
 1   cpu_type           41134 non-null  object 
 2   cpu_limit          41134 non-null  float64
 3   cpu_usage          41134 non-null  float64
 4   ram_usage          41134 non-null  float64
 5   num_req            41134 non-null  int64  
 6   conc_lvl           41134 non-null  int64  
 7   latency            41134 non-null  int64  
 8   hour               41134 non-null  int32  
 9   dayofweek          41134 non-null  int32  
 10  log_cpu_usage      41134 non-null  float64
 11  log_ram_usage      41134 non-null  float64
 12  log_num_req        41134 non-null  float64
 13  log_conc_lvl       41134 non-null  float64
 14  ram_limit_numeric  41134 non-null  float64
dtypes: float64(8), int32(2), int64(4), object(1)
memory usage: 4.4+ MB
Non

In [18]:
df_train.drop(columns=['cpu_type'], inplace=True)
df_test.drop(columns=['cpu_type'], inplace=True)

In [19]:
df_train

,ID,cpu_limit,cpu_usage,ram_usage,num_req,conc_lvl,latency,hour,dayofweek,log_cpu_usage,log_ram_usage,log_num_req,log_conc_lvl,ram_limit_numeric
0,2714,2.0,0.084450,46854144.0,100,71,7429000,23,3,0.081073,17.662550,4.615121,4.276666,70.0
1,38113,2.0,0.277893,55934976.0,300,21,2196000,23,3,0.245213,17.839700,5.707110,3.091042,70.0
2,33409,2.5,0.351042,59109376.0,700,1,37000,13,3,0.300876,17.894900,6.552508,0.693147,66.0
3,10123,0.8,0.026230,43659264.0,100,41,3946000,2,5,0.025892,17.591926,4.615121,3.737670,768.0
4,12815,0.4,0.108346,51048448.0,800,51,5084000,6,3,0.102869,17.748286,6.685861,3.951244,512.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41129,54343,0.7,0.169227,50458624.0,900,11,1039000,2,4,0.156343,17.736664,6.803505,2.484907,896.0
41130,38158,1.5,0.147585,50913280.0,300,1,53000,14,4,0.137660,17.745634,5.707110,0.693147,640.0
41131,860,1.8,0.134895,51859456.0,900,31,3145000,16,2,0.126540,17.764048,6.803505,3.465736,640.0
41132,15795,0.7,0.203862,45862912.0,800,21,2001000,3,3,0.185535,17.641167,6.685861,3.091042,768.0


In [20]:
df_train.shape

(41134, 14)

In [21]:
df_train

,ID,cpu_limit,cpu_usage,ram_usage,num_req,conc_lvl,latency,hour,dayofweek,log_cpu_usage,log_ram_usage,log_num_req,log_conc_lvl,ram_limit_numeric
0,2714,2.0,0.084450,46854144.0,100,71,7429000,23,3,0.081073,17.662550,4.615121,4.276666,70.0
1,38113,2.0,0.277893,55934976.0,300,21,2196000,23,3,0.245213,17.839700,5.707110,3.091042,70.0
2,33409,2.5,0.351042,59109376.0,700,1,37000,13,3,0.300876,17.894900,6.552508,0.693147,66.0
3,10123,0.8,0.026230,43659264.0,100,41,3946000,2,5,0.025892,17.591926,4.615121,3.737670,768.0
4,12815,0.4,0.108346,51048448.0,800,51,5084000,6,3,0.102869,17.748286,6.685861,3.951244,512.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41129,54343,0.7,0.169227,50458624.0,900,11,1039000,2,4,0.156343,17.736664,6.803505,2.484907,896.0
41130,38158,1.5,0.147585,50913280.0,300,1,53000,14,4,0.137660,17.745634,5.707110,0.693147,640.0
41131,860,1.8,0.134895,51859456.0,900,31,3145000,16,2,0.126540,17.764048,6.803505,3.465736,640.0
41132,15795,0.7,0.203862,45862912.0,800,21,2001000,3,3,0.185535,17.641167,6.685861,3.091042,768.0


In [22]:
X = df_train.drop(columns=['ID', 'latency'])
y = df_train['latency']

In [23]:
X.describe()
X.shape

(41134, 12)

In [24]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.metrics import r2_score

pipeline_lgbm = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('lgbm', LGBMRegressor())
])

pipeline_rf = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor())
])

pipeline_xgb = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('xgb', XGBRegressor(subsample = 0.8, n_estimators = 225, max_depth = 7, learning_rate = 0.1, colsample_bytree = 0.8))
])



In [26]:
pipeline_lgbm.fit(X_train, y_train)
# pipeline_rf.fit(X_train, y_train)
pipeline_xgb.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007323 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16096
[LightGBM] [Info] Number of data points in the train set: 32907, number of used features: 90
[LightGBM] [Info] Start training from score 6489539.216337


Pipeline(steps=[('poly_features', PolynomialFeatures(include_bias=False)),
                ('scaler', StandardScaler()),
                ('xgb',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=0.8, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=7, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=225, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [104]:
# Best Model from RandomizedSearchCV
# best_xgb_model = random_search.best_estimator_
# print(random_search.best_params_)

{'xgb__subsample': 0.8, 'xgb__n_estimators': 275, 'xgb__max_depth': 7, 'xgb__learning_rate': 0.05, 'xgb__colsample_bytree': 0.7}


In [27]:
voting_regressor = VotingRegressor([
    ('lgbm', pipeline_lgbm),
    # ('rf', pipeline_rf),
    ('xgb', pipeline_xgb)
])

voting_regressor.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008619 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16096
[LightGBM] [Info] Number of data points in the train set: 32907, number of used features: 90
[LightGBM] [Info] Start training from score 6489539.216337


VotingRegressor(estimators=[('lgbm',
                             Pipeline(steps=[('poly_features',
                                              PolynomialFeatures(include_bias=False)),
                                             ('scaler', StandardScaler()),
                                             ('lgbm', LGBMRegressor())])),
                            ('xgb',
                             Pipeline(steps=[('poly_features',
                                              PolynomialFeatures(include_bias=False)),
                                             ('scaler', StandardScaler()),
                                             ('xgb',
                                              XGBRegressor(base_score=None,
                                                           booster=None,
                                                           callbacks=None,
                                                           colsample_byl...
                                                           grow_policy=None,
                                                           importance_type=None,
                                                           interaction_constraints=None,
                                                           learning_rate=0.1,
                                                           max_bin=None,
                                                           max_cat_threshold=None,
                                                           max_cat_to_onehot=None,
                                                           max_delta_step=None,
                                                           max_depth=7,
                                                           max_leaves=None,
                                                           min_child_weight=None,
                                                           missing=nan,
                                                           monotone_constraints=None,
                                                           multi_strategy=None,
                                                           n_estimators=225,
                                                           n_jobs=None,
                                                           num_parallel_tree=None,
                                                           random_state=None, ...))]))])

In [29]:
y_val_pred = voting_regressor.predict(X_val)
r2_xgb = r2_score(y_val, y_val_pred)
print(f'R-squared (R²) on Validation Set with Best lgbm: {r2_xgb}')

R-squared (R²) on Validation Set with Best lgbm: 0.9692335745504258


In [112]:
# y_val_pred = model.predict(X_val)
# r2 = r2_score(y_val, y_val_pred)
# print(f'R-squared (R²) on Validation Set: {r2}')

In [32]:
voting_regressor = VotingRegressor([
    ('lgbm', pipeline_lgbm),
    # ('rf', pipeline_rf),
    ('xgb', pipeline_xgb)
])

voting_regressor.fit(X, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009179 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16106
[LightGBM] [Info] Number of data points in the train set: 41134, number of used features: 90
[LightGBM] [Info] Start training from score 6483698.181553


VotingRegressor(estimators=[('lgbm',
                             Pipeline(steps=[('poly_features',
                                              PolynomialFeatures(include_bias=False)),
                                             ('scaler', StandardScaler()),
                                             ('lgbm', LGBMRegressor())])),
                            ('xgb',
                             Pipeline(steps=[('poly_features',
                                              PolynomialFeatures(include_bias=False)),
                                             ('scaler', StandardScaler()),
                                             ('xgb',
                                              XGBRegressor(base_score=None,
                                                           booster=None,
                                                           callbacks=None,
                                                           colsample_byl...
                                                           grow_policy=None,
                                                           importance_type=None,
                                                           interaction_constraints=None,
                                                           learning_rate=0.1,
                                                           max_bin=None,
                                                           max_cat_threshold=None,
                                                           max_cat_to_onehot=None,
                                                           max_delta_step=None,
                                                           max_depth=7,
                                                           max_leaves=None,
                                                           min_child_weight=None,
                                                           missing=nan,
                                                           monotone_constraints=None,
                                                           multi_strategy=None,
                                                           n_estimators=225,
                                                           n_jobs=None,
                                                           num_parallel_tree=None,
                                                           random_state=None, ...))]))])

In [33]:
X_train

,cpu_limit,cpu_usage,ram_usage,num_req,conc_lvl,hour,dayofweek,log_cpu_usage,log_ram_usage,log_num_req,log_conc_lvl,ram_limit_numeric
38307,0.1,0.096577,51159040.0,100,51,14,4,0.092194,17.750450,4.615121,3.951244,1024.0
22788,0.6,0.098265,50835456.0,1000,41,4,3,0.093732,17.744105,6.908755,3.737670,256.0
6010,2.0,0.560225,58814464.0,800,11,4,5,0.444830,17.889898,6.685861,2.484907,66.0
32313,0.2,0.193234,63541248.0,800,11,3,0,0.176667,17.967200,6.685861,2.484907,512.0
33689,1.5,0.000000,35258368.0,100,11,18,3,0.000000,17.378213,4.615121,2.484907,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6265,0.2,0.195358,53153792.0,600,31,7,4,0.178446,17.788700,6.398595,3.465736,384.0
11284,1.1,0.098054,52637696.0,800,51,23,2,0.093540,17.778943,6.685861,3.951244,896.0
38158,2.5,0.233490,48197632.0,200,51,12,3,0.209848,17.690820,5.303305,3.951244,66.0
860,3.0,0.370481,58646528.0,700,61,10,6,0.315162,17.887039,6.552508,4.127134,73.0


In [34]:
df_test

,ID,timestamp,cpu_limit,cpu_usage,ram_usage,num_req,conc_lvl,ram_limit_numeric
0,1182,2022-03-10 11:28:06,0.1,0.100592,60809216.0,900,51,1024.0
1,19002,2022-03-11 02:33:23,0.2,0.202658,64212992.0,800,71,1024.0
2,13933,2022-03-06 11:38:14,0.1,0.100553,68841472.0,900,41,896.0
3,10049,2022-03-17 00:09:05,1.0,0.186460,49979392.0,700,21,896.0
4,19412,2021-11-11 10:55:26,2.5,0.369512,58982400.0,1000,81,68.0
...,...,...,...,...,...,...,...,...
17624,741,2021-11-13 20:13:24,4.0,0.713527,62636032.0,800,11,71.0
17625,36105,2022-03-06 13:56:43,0.1,0.100533,56094720.0,700,11,896.0
17626,25313,2022-03-19 02:01:13,0.9,0.197724,48177152.0,700,21,512.0
17627,25044,2022-03-19 10:25:10,0.2,0.200379,50946048.0,800,11,128.0


In [35]:
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
df_test['hour'] = df_test['timestamp'].dt.hour
df_test['day'] = df_test['timestamp'].dt.day
df_test['month'] = df_test['timestamp'].dt.month
df_test['dayofweek'] = df_test['timestamp'].dt.dayofweek

df_test.drop(columns=['timestamp'], inplace=True)

In [36]:
df_test['log_cpu_usage'] = np.log1p(df_test['cpu_usage'])
df_test['log_ram_usage'] = np.log1p(df_test['ram_usage'])
df_test['log_num_req'] = np.log1p(df_test['num_req'])
df_test['log_conc_lvl'] = np.log1p(df_test['conc_lvl'])

In [37]:
df_test

,ID,cpu_limit,cpu_usage,ram_usage,num_req,conc_lvl,ram_limit_numeric,hour,day,month,dayofweek,log_cpu_usage,log_ram_usage,log_num_req,log_conc_lvl
0,1182,0.1,0.100592,60809216.0,900,51,1024.0,11,10,3,3,0.095848,17.923252,6.803505,3.951244
1,19002,0.2,0.202658,64212992.0,800,71,1024.0,2,11,3,4,0.184534,17.977716,6.685861,4.276666
2,13933,0.1,0.100553,68841472.0,900,41,896.0,11,6,3,6,0.095813,18.047317,6.803505,3.737670
3,10049,1.0,0.186460,49979392.0,700,21,896.0,0,17,3,3,0.170974,17.727121,6.552508,3.091042
4,19412,2.5,0.369512,58982400.0,1000,81,68.0,10,11,11,3,0.314454,17.892750,6.908755,4.406719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17624,741,4.0,0.713527,62636032.0,800,11,71.0,20,13,11,5,0.538554,17.952851,6.685861,2.484907
17625,36105,0.1,0.100533,56094720.0,700,11,896.0,13,6,3,6,0.095795,17.842552,6.552508,2.484907
17626,25313,0.9,0.197724,48177152.0,700,21,512.0,2,19,3,5,0.180423,17.690395,6.552508,3.091042
17627,25044,0.2,0.200379,50946048.0,800,11,128.0,10,19,3,5,0.182637,17.746278,6.685861,2.484907


In [38]:
df_test = df_test.set_index('ID')
missing_cols = set(X_train.columns) - set(df_test.columns)
for col in missing_cols:
    df_test[col] = 0
df_test = df_test.reset_index()
X_test = df_test[X_train.columns]

In [39]:
X_test.shape

(17629, 12)

In [41]:
y_test_pred = voting_regressor.predict(X_test)

In [43]:
df_test

,ID,cpu_limit,cpu_usage,ram_usage,num_req,conc_lvl,ram_limit_numeric,hour,day,month,dayofweek,log_cpu_usage,log_ram_usage,log_num_req,log_conc_lvl
0,1182,0.1,0.100592,60809216.0,900,51,1024.0,11,10,3,3,0.095848,17.923252,6.803505,3.951244
1,19002,0.2,0.202658,64212992.0,800,71,1024.0,2,11,3,4,0.184534,17.977716,6.685861,4.276666
2,13933,0.1,0.100553,68841472.0,900,41,896.0,11,6,3,6,0.095813,18.047317,6.803505,3.737670
3,10049,1.0,0.186460,49979392.0,700,21,896.0,0,17,3,3,0.170974,17.727121,6.552508,3.091042
4,19412,2.5,0.369512,58982400.0,1000,81,68.0,10,11,11,3,0.314454,17.892750,6.908755,4.406719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17624,741,4.0,0.713527,62636032.0,800,11,71.0,20,13,11,5,0.538554,17.952851,6.685861,2.484907
17625,36105,0.1,0.100533,56094720.0,700,11,896.0,13,6,3,6,0.095795,17.842552,6.552508,2.484907
17626,25313,0.9,0.197724,48177152.0,700,21,512.0,2,19,3,5,0.180423,17.690395,6.552508,3.091042
17627,25044,0.2,0.200379,50946048.0,800,11,128.0,10,19,3,5,0.182637,17.746278,6.685861,2.484907


In [44]:
submission = pd.DataFrame({'ID': df_test['ID'], 'latency': y_test_pred})
submission.to_csv('submission_voting_2.csv', index=False)

print(submission.head())

      ID       latency
0   1182  1.095707e+08
1  19002  6.117965e+07
2  13933  1.934923e+07
3  10049  2.054799e+06
4  19412  7.764894e+06
